### Graph Construction

In [1]:
import pickle
import pandas as pd
import inspect

# save file path
nodes_path = './data/graph/nodes/'
edges_path = './data/graph/edges/'

def read_file(file):
    with open(file, encoding='utf-8', errors='ignore') as f:
        data = f.readlines()
    return data

def read_dict(file_path):
    f_read = open(file_path, 'rb')
    dict = pickle.load(f_read)
    f_read.close()
    return dict

def save_dict(dict, file_path):
    f_save = open(file_path, 'wb')
    pickle.dump(dict, f_save)
    f_save.close()


def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

#### Nodes

In [ ]:
import re

target = []  # gene symbol
drug = []  # drug_name
pathway = []  # Reactome_id
disease = []  # Indication_ICD-11
# herb = []  # HERB_herb_id

# nodes_Drug
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)
for k,v in drug_id2name_dict.items():
    if v not in drug:
        drug.append(v)


# node_Target
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)
for k,v in target_id2name_dict.items():
    if v not in target:
        target.append(v)

target2drug_file = './data/target2drug.xlsx'
target2drug_df = pd.read_excel(target2drug_file)
targets = target2drug_df.TargetID.tolist()
drugs_id = target2drug_df.DrugID.tolist()
# pattern = r"T\d"
for tid in targets:
    if tid in target_id2name_dict:
        tg = target_id2name_dict[tid]
    # elif re.match(pattern, tid):
    #     tg = 'ttd:'+tid
    else:
        tg = tid
    if tg not in target:
        target.append(tg)
for did in drugs_id:
    if did in drug_id2name_dict:
        dg = drug_id2name_dict[did]
        if dg not in drug:
            drug.appeng(dg)


# nodes_pathway
target2pathway_file = './data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
for key in target2pathway_dict:
    if key in target_id2name_dict:
        tg = target_id2name_dict[key]
    else:
        tg = key
    if tg not in target:
        target.append(tg)
    for p in target2pathway_dict[key]:
        if p not in pathway:
            pathway.append(p)


# nodes_Disease
target2disease_file = './data/target2disease.pkl'
target2disease_dict = read_dict(target2disease_file)
for key in target2disease_dict:
    if key in target_id2name_dict:
        tg = target_id2name_dict[key]
    # elif re.match(pattern, tid):
    #     tg = 'ttd:'+tid
    else:
        tg = key
    if tg not in target:
        target.append(tg)
    for dis in target2disease_dict[key]:
        if dis not in disease:
            disease.append(dis)

# nodes_Drug
drug2disease_file = './data/drug2disease.pkl'
drug2disease_dict = read_dict(drug2disease_file)
for dg in drug2disease_dict:
    if dg not in drug:
        drug.append(dg)
    for dis in drug2disease_dict[dg]:
        if dis not in disease:
            disease.append(dis)

Add AP data

In [3]:
ingredient_id = pd.read_excel('./data/case/AP/ingredient-ttd-drugbank.xlsx')

ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')
diseases = ['AP']
icds = ['[ICD-11: DC31]']

compounds_gene_file = './data/case/AP/compounds-gene/'
    
for i in range(1,23):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')
    
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        if g not in target:
            target.append(g)

for dis in diseases:
    dis2tg_df = pd.read_excel(f'./data/case/AP/disease-gene-{dis}.xlsx')
    gene_symbol = dis2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        if g not in target:
            target.append(g)

# add Drug,Disease nodes of case
for dg in drugs:
    if dg not in drug:
        drug.append(dg)

for i in range(len(diseases)):
    dis_w_icd = f'{diseases[i]} {icds[i]}'
    if dis_w_icd not in disease:
        disease.append(dis_w_icd)
# for dis in diseases:
#     if dis not in disease:
#         disease.append(dis)

##### Add AP(sub) data

In [ ]:
ingredient_id = pd.read_excel('./data/case/AP(sub)/ingredient-ttd-drugbank.xlsx')

ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')
diseases = ['SAP', 'MAP']
# icds = ['[ICD-11: DC31-S]', '[ICD-11: DC31-M]']

compounds_gene_file = './data/case/AP(sub)/compounds-gene/'
    
for i in range(1,23):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')
    
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        if g not in target:
            target.append(g)

for dis in diseases:
    dis2tg_df = pd.read_excel(f'./data/case/AP/disease-gene-{dis}.xlsx')
    gene_symbol = dis2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        if g not in target:
            target.append(g)

# add Drug,Disease nodes of case
for dg in drugs:
    if dg not in drug:
        drug.append(dg)

for dis in diseases:
    if dis not in disease:
        disease.append(dis)

In [ ]:
# Add AP data (save CQCQT as Herb)
'''
target = []  # TTD_target_id
drug = []  # drug_name
pathway = []  # Reactome_id
disease = []  # Indication_ICD-11
herb = []  # HERB_herb_id
formula = []  # PinYin
'''
# # AP_formula
# formula.append('ChaiQinChengQiTang')

# # save CQCQT as Herb
# if 'ChaiQinChengQiTang' not in herb:
#     herb.append('ChaiQinChengQiTang')

# save CQCQD as Drug
if 'CQCQD' not in drug:
    drug.append('CQCQD')

# AP_disease
mAP = 'mild  acute pancreatitis [ICD-11: DC31]'
sAP = 'severe acute pancreatitis [ICD-11: DC31]'
if mAP not in disease:
    disease.append(mAP)
if sAP not in disease:
    disease.append(sAP)

# AP_ingredient-gene
i2g_df = pd.read_csv('./data/case/AP/ingredient-gene.csv')
ingre_name = i2g_df['ingredient name'].to_list()
for ingre in ingre_name:
    if ingre not in drug:
        drug.append(ingre)

ingre_gene_symbol = i2g_df['gene symbol'].to_list()
# AP_disease-gene
map_d2g_df = pd.read_csv('./data/case/AP/disease-gene-MAP.csv')
sap_d2g_df = pd.read_csv('./data/case/AP/disease-gene-SAP.csv')
map_gene_symbol = map_d2g_df['gene symbol'].to_list()
sap_gene_symbol = sap_d2g_df['gene symbol'].to_list()

gene_symbol = ingre_gene_symbol + map_gene_symbol + sap_gene_symbol
target_id2name_dict = read_dict('./data/target_ttdid2name.pkl')
t_name2id_dict = {v:k for k,v in target_id2name_dict.items()}
# target_names = [v for k,v in target_id2name_dict.items()]
for g in gene_symbol:
    if g in t_name2id_dict.keys():
        if t_name2id_dict[g] not in target:
            target.append(t_name2id_dict[g])
    else:
        if g not in target:
            target.append(g)


##### Add Asthma,T2MD, T2DM data

In [ ]:
'''
target = []  # TTD_target_id
drug = []  # drug_name
pathway = []  # Reactome_id
disease = []  # Indication_ICD-11
herb = []  # HERB_herb_id
formula = []  # PinYin
'''
preprocessing = ['ML', 'RWR', 'DEG']
LP_rels = ['Asthma-Theophylline', 'T2DM-Acarbose', 'T2DM-Glimepiride']
drugs = ['Theophylline', 'Acarbose', 'Glimepiride']
diseases = ['Asthma', 'T2DM', 'T2DM']
icds = ['[ICD-11: CA23]', '[ICD-11: 5A11]', '[ICD-11: 5A11]']

case_data_file = './data/case/'

for i in range(len(preprocessing)):
    preprocess = preprocessing[i]       
    for j in range(len(LP_rels)):
        LP_rel = LP_rels[j]
        dis = diseases[j]
        dg = drugs[j]
        # add Target nodes of case
        i2g_df = pd.read_csv(case_data_file+f'{preprocess}/{LP_rel}/ingredient-gene-{dg}.csv')
        d2g_df = pd.read_csv(case_data_file+f'{preprocess}/{LP_rel}/disease-gene-{dis}.csv')
        ingre_gene_symbol = i2g_df['gene symbol'].to_list()
        dis_gene_symbol = d2g_df['gene symbol'].to_list()
        gene_symbol = ingre_gene_symbol + dis_gene_symbol
        target_id2name_dict = read_dict('./data/target_ttdid2name.pkl')
        t_name2id_dict = {v:k for k,v in target_id2name_dict.items()}
        for g in gene_symbol:
            if g in t_name2id_dict.keys():
                if t_name2id_dict[g] not in target:
                    target.append(t_name2id_dict[g])
            else:
                if g not in target:
                    target.append(g)
        # add Drug,Disease nodes of case
        # if dis in 'Asthma [ICD-11: CA23]':
        #     dis = 'Asthma [ICD-11: CA23]'
        # if dis not in disease:
        #     disease.append(dis)
        if dg not in drug:
            drug.append(dg)

##### Add TRQ data

In [4]:
'''
target = []  # gene_symbol
drug = []  # drug_name
pathway = []  # Reactome_id
disease = []  # Indication_ICD-11
herb = []  # HERB_herb_id
formula = []  # PinYin
'''
diseases = ['CRKP', 'KP']
icds = ['[ICD-11: MG50.54]', '[ICD-11: CA40.03]']
case_data_file = './data/case/TRQ/'

ingredient_id = pd.read_excel(case_data_file+'/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('TRQ')

for dg in drugs:
    if dg not in drug:
        drug.append(dg)

compounds_gene_file = case_data_file+'/compounds-gene/'
for i in range(1,27):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')    
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        if g not in target:
            target.append(g)

target_id2name_dict = read_dict('./data/target_ttdid2name.pkl')
t_name2id_dict = {v:k for k,v in target_id2name_dict.items()}
     
for i in range(len(diseases)):
    dis = diseases[i]
    # add Target nodes of case    
    dis2g_df = pd.read_excel(case_data_file+f'disease-gene-{dis}.xlsx')    
    dis_gene_symbol = dis2g_df['gene symbol'].to_list()
    for g in dis_gene_symbol:
        if g not in target:
            target.append(g)
    # add Drug,Disease nodes of case
    # if dis in 'Asthma [ICD-11: CA23]':
    #     dis = 'Asthma [ICD-11: CA23]'
# for dis in diseases:
    # if dis not in disease:
    #     disease.append(dis)
for i in range(len(diseases)):
    dis_w_icd = f'{diseases[i]} {icds[i]}'
    if dis_w_icd not in disease:
        disease.append(dis_w_icd)  
    

##### save_nodes

In [5]:
icd2name_dict = {}
dis_name2icd_dict = {}
for s in disease:
    line = s.split(' [')
    if len(line)>1:
        dis = line[0].strip()
        icd = '[' + line[1].strip()
        if icd not in icd2name_dict:
            icd2name_dict[icd]=[]
        icd2name_dict[icd].append(dis)
        dis_name2icd_dict[dis] = icd
    else:
        icd2name_dict[s] = s
        dis_name2icd_dict[s] = s
save_dict(dis_name2icd_dict, './data/disease2icd.pkl')
save_dict(icd2name_dict, './data/icd2disease.pkl')
icd11 = list(icd2name_dict.keys())

In [6]:
target_dict, drug_dict, pathway_dict, disease_dict, icd11_dict = {}, {}, {}, {}, {}

node_list = [target, drug, pathway, disease, icd11]
dict_list = [target_dict, drug_dict, pathway_dict, disease_dict, icd11_dict]

for index in range(len(node_list)):
    nodes = node_list[index]
    dic = dict_list[index]
    var_name = retrieve_name(dic)[0]
    for i in range(len(nodes)):
        dic[nodes[i]] = i
    save_path = nodes_path + var_name + '.pkl'
    save_dict(dic, save_path)

#### Edges

In [7]:
target2pathway_file = './data/target2pathway_KEGG.pkl'  #(_Reactome/KEGG/all)
target2disease_file = './data/target2disease.pkl'
target2drug_file = './data/target2drug.xlsx'
drug2disease_file = './data/drug2disease.pkl'
gt_drug2disease_file = './data/groundtruth_drug2disease.pkl'  # ground-truth

node_target = read_dict('./data/graph/nodes/target_dict.pkl')  # gene_symbol
node_drug = read_dict('./data/graph/nodes/drug_dict.pkl')  # drug_name
node_pathway = read_dict('./data/graph/nodes/pathway_dict.pkl')  # Reactome_id
node_disease = read_dict('./data/graph/nodes/disease_dict.pkl')  # Indication_ICD-11
node_icd = read_dict('./data/graph/nodes/icd11_dict.pkl')  # icd11_code

In [ ]:
# Query the node id of the AP-related objects

mAP = 'mild acute pancreatitis [ICD-11: DC31]'
sAP = 'severe acute pancreatitis [ICD-11: DC31]'
formula = 'ChaiQinChengQiTang'

node_disease[mAP],node_disease[sAP],node_drug[formula]

In [ ]:
# Query the node id of the specified objects
'''
preprocessing = ['ML', 'RWR']
LP_rels = ['Asthma-Theophylline', 'T2MD-Acarbose']

drugs = ['Theophylline', 'Acarbose']
diseases = ['Asthma', 'T2MD']
# Asthma [ICD-11: CA23]
# Type 2 diabetes mellitus [ICD-11: 5A11]
'''
# diseases = ['Asthma [ICD-11: CA23]', 'T2MD']
# for dis in diseases:
#     print(dis, ':', node_disease[dis])
icd11 = ['[ICD-11: CA23]', '[ICD-11: 5A11]'] # ['Asthma', 'T2MD']
for icd in icd11:
    print(icd, ':', node_icd[icd])

drugs = ['Theophylline', 'Acarbose', 'Glimepiride']
for dg in drugs:
    print(dg, ':', node_drug[dg])

Drug-Disease(+AP)

In [ ]:
# import scipy.misc
# from scipy import io
import numpy as np
import pandas as pd
import hdf5storage as hg

# drug_id2name_dict
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)

# target_id2name_dict
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)

# disease_icd2name_dict
disease_icd2name_file = './data/icd2disease.pkl'
disease_icd2name_dict = read_dict(disease_icd2name_file)

# edges_target2drug
target2drug_file = './data/target2drug_addAP.xlsx'
t2dg_df = pd.read_excel(target2drug_file)
t2dg_adj = np.zeros((len(node_target), len(node_drug)))
for index, data in t2dg_df.iterrows():
    tid = data[0]    
    drug_id = data[1]
    if tid in target_id2name_dict:
        target = target_id2name_dict[tid]
    else:
        target = tid
    if drug_id in drug_id2name_dict:
        drug = drug_id2name_dict[drug_id]
        u = node_target[target]
        v = node_drug[drug]
        t2dg_adj[u][v] = 1
# CQCQT
CQCQT2target_file = './data/CQCQT2target.pkl'
CQCQT2target_dict = read_dict(CQCQT2target_file)
for key, values in CQCQT2target_dict.items():
    v = node_drug[key]
    for tid in values:
        if tid in target_id2name_dict:
            target = target_id2name_dict[tid]
        else:
            target = tid
        u = node_target[target]
        t2dg_adj[u][v] = 1
# np.save(edges_path+'target2drug.npy', t2dg_adj)
print('completed: target2drug.npy')

# edges_target2pathway
target2pathway_file = f'./data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
t2p_adj = np.zeros((len(node_target), len(node_pathway)))
for key in target2pathway_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key        
    u = node_target[target]
    for pathway in target2pathway_dict[key]:
        v = node_pathway[pathway]
        t2p_adj[u][v] = 1
# np.save(edges_path+f'target2pathway.npy', t2p_adj)
print(f'completed: target2pathway.npy')


# edges_target2disease
target2disease_file = './data/target2disease.pkl'
target2disease_dict = read_dict(target2disease_file)
t2d_adj = np.zeros((len(node_target), len(node_disease)))
'''
print(t2d_adj.shape)
u = node_target['PLSCR1']
mAP = 'mild acute pancreatitis [ICD-11: DC31]'
sAP = 'severe acute pancreatitis [ICD-11: DC31]'
v1 = node_disease[mAP]
v2 = node_disease[sAP]
print(t2d_adj[u][v1],t2d_adj[u][v2])    # 0.0, 0.0
'''
for key in target2disease_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key
    u = node_target[target]
    for disease in target2disease_dict[key]:
        v = node_disease[disease]
        t2d_adj[u][v] = 1    
# np.save(edges_path+'target2disease.npy', t2d_adj)
print('completed: target2disease.npy')
''''''
u = node_target['PLSCR1']
mAP = 'mild acute pancreatitis [ICD-11: DC31]'
sAP = 'severe acute pancreatitis [ICD-11: DC31]'
v1 = node_disease[mAP]
v2 = node_disease[sAP]
print(target2disease_dict['PLSCR1'])
print(t2d_adj[u][v1],t2d_adj[u][v2])    # 0.0, 1.0
''''''

# edges_drug2disease
drug2disease_file = './data/drug2disease.pkl'
drug2disease_dict = read_dict(drug2disease_file)
dg2d_adj = np.zeros((len(node_drug), len(node_disease)))
for key in drug2disease_dict:
    drug = key
    u = node_drug[drug]
    for disease in drug2disease_dict[key]:
        v = node_disease[disease]
        dg2d_adj[u][v] = 1
# np.save(edges_path+'ingredient2disease.npy', dg2d_adj)
print('completed: ingredient2disease.npy')

# ground-truth_drug2disease
groundtruth_file = './data/groundtruth/gt_drug2disease+AP.pkl'
gt_dict = read_dict(groundtruth_file)
gt_dg2d_adj = np.zeros((len(node_drug), len(node_disease)))
for dg, dis in gt_dict.items():
    u = node_drug[dg]
    for d in dis:
        v = node_disease[d]
        gt_dg2d_adj[u][v] = 1
np.save(edges_path+'groundtruth_drug2disease.npy', gt_dg2d_adj)
print('completed: ground-truth_drug2disease.npy')


graph_path = './data/graph/mat/'
hg.savemat(graph_path+'woHERB_AP.mat', {"TvsD":t2d_adj, "TvsI":t2dg_adj, "IvsD":dg2d_adj, "IvsD_gt":gt_dg2d_adj, "TvsP":t2p_adj})
print('completed: matlab file')

Drug-ICD11 (+APsub)

In [ ]:
# import scipy.misc
# from scipy import io
import numpy as np
import pandas as pd
import hdf5storage as hg
import torch

nodes_path = './data/case/AP/nodes/'
node_target = read_dict(nodes_path+'target_dict.pkl')  # gene symbol
node_drug = read_dict(nodes_path+'drug_dict.pkl')  # drug_name
node_pathway = read_dict(nodes_path+'pathway_dict.pkl')  # Reactome_id
node_disease = read_dict(nodes_path+'disease_dict.pkl')  # Indication_ICD-11
node_icd = read_dict(nodes_path+'icd11_dict.pkl')  # icd11_code

# drug_id2name_dict
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)

# target_id2name_dict
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)

# disease_icd2name_dict
disease_name2icd_file = './data/disease2icd.pkl'
disease_name2icd_dict = read_dict(disease_name2icd_file)

# edges_target2pathway
target2pathway_file = f'./data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
t2p_adj = np.zeros((len(node_target), len(node_pathway)))
for key in target2pathway_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key        
    u = node_target[target]
    for pathway in target2pathway_dict[key]:
        v = node_pathway[pathway]
        t2p_adj[u][v] = 1
# np.save(edges_path+f'target2pathway.npy', t2p_adj)
print(f'completed: target2pathway.npy')

    
# edges_target2drug
target2drug_file = './data/target2drug.xlsx'
t2dg_df = pd.read_excel(target2drug_file)
t2dg_adj = np.zeros((len(node_target), len(node_drug)))
for index, data in t2dg_df.iterrows():
    tid = data[0]    
    drug_id = data[1]
    if tid in target_id2name_dict:
        target = target_id2name_dict[tid]
    else:
        target = tid
    if drug_id in drug_id2name_dict:
        drug = drug_id2name_dict[drug_id]
        u = node_target[target]
        v = node_drug[drug]
        t2dg_adj[u][v] = 1
# np.save(edges_path+'target2drug.npy', t2dg_adj)
print('completed: target2drug.npy')


# edges_target2icd_gt
target2disease_file = './data/groundtruth/gt_target2disease.pkl'
target2disease_dict = read_dict(target2disease_file)
t2icd_adj = np.zeros((len(node_target), len(node_icd)))
for key in target2disease_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key
    u = node_target[target]
    for disease in target2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        t2icd_adj[u][v] = 1    
# np.save(edges_path+'target2disease.npy', t2d_adj)
print('completed: target2disease.npy')


# edges_drug2disease_gt
drug2disease_file = './data/groundtruth/gt_drug2disease.pkl'
drug2disease_dict = read_dict(drug2disease_file)
dg2d_adj = np.zeros((len(node_drug), len(node_icd)))
for key in drug2disease_dict:
    drug = key
    u = node_drug[drug]
    for disease in drug2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        dg2d_adj[u][v] = 1
# np.save(edges_path+'drug2disease.npy', dg2d_adj)
print('completed: drug2disease.npy')


# add AP data
ingredient_id = pd.read_excel('./data/case/AP/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')
# add target-drug
for i in range(1,23):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')
    v = node_drug[compounds]
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2dg_adj[u][v] = 1
CQCQD2tg_df = pd.read_excel('./data/case/AP/CQCQD-gene.xlsx')
v = node_drug['CQCQD']
gene_symbol = compounds2tg_df['gene symbol'].to_list()
for g in gene_symbol:
    u = node_target[g]
    t2dg_adj[u][v] = 1

# add target-disease
diseases = ['SAP', 'MAP']
for dis in diseases:
    v = node_icd[dis]
    dis2tg_df = pd.read_excel(f'./data/case/AP/disease-gene-{dis}.xlsx')
    gene_symbol = dis2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2icd_adj[u][v] = 1

# add drug-disease
for dg in drugs:
    u = node_drug[dg]
    for dis in diseases:
        v = node_icd[dis]
        dg2d_adj[u][v] = 1

data = {"TvsD":t2icd_adj, "TvsI":t2dg_adj, "IvsD":dg2d_adj, "TvsPK":t2p_adj}
# graph_path = './data/graph/mat/'
# hg.savemat(graph_path+'woHERB_AP.mat', {"TvsD":t2icd_adj, "TvsI":t2dg_adj, "IvsD":dg2d_adj, "gtIvsD":dg2d_gt_adj,
#                                         "TvsP":t2p_adj})
# print('completed: matlab file')

In [ ]:
import dgl
from dgl.data.utils import save_graphs

G = dgl.heterograph({
        ('target1', 'included-by', 'pathway') : data['TvsPK'].nonzero(),
        ('pathway', 'includes', 'target2') : data['TvsPK'].transpose().nonzero(),
        ('target2', 'causes', 'disease') : data['TvsD'].nonzero(),
        ('drug', 'aims-at', 'target1') : data['TvsI'].transpose().nonzero(),
        ('drug', 'treats', 'disease') : data['IvsD'].nonzero(),
    },
    num_nodes_dict = {'disease': len(node_icd), 'drug': len(node_drug),
                      'target1': len(node_target), 'target2': len(node_target),
                      'pathway': len(node_pathway)}
    )

save_graphs(f'./datasets/woHERB_KEGG_AP', G)
print(G)

In [ ]:
# Genenrat the query pair nids

drugs_nids = []
diseases_nids = []

ingredient_id = pd.read_excel('./data/case/AP/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')

for dg in drugs:
    drugs_nids.append(node_drug[dg])
# add target-disease
diseases = ['SAP', 'MAP']
for dis in diseases:
    diseases_nids.append(node_icd[dis])

# query_src_nids, query_tgt_nids = torch.tensor([11250, 13788, 11481]), torch.tensor([165, 5, 165])
query_src_nids, query_tgt_nids = torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)
for tgt_nid in diseases_nids:
    for src_nid in drugs_nids:
        query_src_nids = torch.cat((query_src_nids, torch.tensor([src_nid])), dim=0)
        query_tgt_nids = torch.cat((query_tgt_nids, torch.tensor([tgt_nid])), dim=0)

torch.save((query_src_nids, query_tgt_nids), './query_nids/AP_query_nids')

Drug-ICD11 (+AP+TRQ)

In [10]:
import numpy as np
import pandas as pd

nodes_path = './data/graph/nodes/'
node_target = read_dict(nodes_path+'target_dict.pkl')  # gene symbol
node_drug = read_dict(nodes_path+'drug_dict.pkl')  # drug_name
node_pathway = read_dict(nodes_path+'pathway_dict.pkl')  # Reactome_id
node_disease = read_dict(nodes_path+'disease_dict.pkl')  # Indication_ICD-11
node_icd = read_dict(nodes_path+'icd11_dict.pkl')  # icd11_code
edges_path = './data/graph/edges/'

# drug_id2name_dict
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)

# target_id2name_dict
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)

# disease_icd2name_dict
disease_name2icd_file = './data/disease2icd.pkl'
disease_name2icd_dict = read_dict(disease_name2icd_file)

# edges_target2pathway
target2pathway_file = f'./data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
t2p_adj = np.zeros((len(node_target), len(node_pathway)))
for key in target2pathway_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key        
    u = node_target[target]
    for pathway in target2pathway_dict[key]:
        v = node_pathway[pathway]
        t2p_adj[u][v] = 1
# np.save(edges_path+f'target2pathway.npy', t2p_adj)
print(f'completed: target2pathway.npy')

    
# edges_target2drug
target2drug_file = './data/target2drug.xlsx'
t2dg_df = pd.read_excel(target2drug_file)
t2dg_adj = np.zeros((len(node_target), len(node_drug)))
for index, data in t2dg_df.iterrows():
    tid = data[0]    
    drug_id = data[1]
    if tid in target_id2name_dict:
        target = target_id2name_dict[tid]
    else:
        target = tid
    if drug_id in drug_id2name_dict:
        drug = drug_id2name_dict[drug_id]
        u = node_target[target]
        v = node_drug[drug]
        t2dg_adj[u][v] = 1
# np.save(edges_path+'target2drug.npy', t2dg_adj)
print('completed: target2drug.npy')


# edges_target2icd_gt
target2disease_file = './data/groundtruth/gt_target2disease.pkl'
target2disease_dict = read_dict(target2disease_file)
t2icd_adj = np.zeros((len(node_target), len(node_icd)))
for key in target2disease_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key
    u = node_target[target]
    for disease in target2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        t2icd_adj[u][v] = 1    
# np.save(edges_path+'target2disease.npy', t2d_adj)
print('completed: target2disease.npy')


# edges_drug2disease_gt
drug2disease_file = './data/groundtruth/gt_drug2disease.pkl'
drug2disease_dict = read_dict(drug2disease_file)
dg2d_adj = np.zeros((len(node_drug), len(node_icd)))
for key in drug2disease_dict:
    drug = key
    u = node_drug[drug]
    for disease in drug2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        dg2d_adj[u][v] = 1
# np.save(edges_path+'drug2disease.npy', dg2d_adj)
print('completed: drug2disease.npy')

case_data_file = './data/case/' 
## add AP data
case = 'AP'
ingredient_id = pd.read_excel(case_data_file+f'{case}/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')
# add target-drug
compounds_gene_file = case_data_file+f'{case}/compounds-gene/'   
for i in range(1,23):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')
    v = node_drug[compounds]
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2dg_adj[u][v] = 1
CQCQD2tg_df = pd.read_excel(case_data_file+f'{case}/CQCQD-gene.xlsx')
v = node_drug['CQCQD']
gene_symbol = compounds2tg_df['gene symbol'].to_list()
for g in gene_symbol:
    u = node_target[g]
    t2dg_adj[u][v] = 1
# add target-disease
diseases = ['AP']
icds = ['[ICD-11: DC31]']
for dis in diseases:
    i = diseases.index(dis)
    v = node_icd[icds[i]]
    dis2tg_df = pd.read_excel(case_data_file+f'{case}/disease-gene-{dis}.xlsx')
    gene_symbol = dis2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2icd_adj[u][v] = 1
# add drug-disease
for dg in drugs:
    u = node_drug[dg]
    for dis in icds:
        v = node_icd[dis]
        dg2d_adj[u][v] = 1

## add TRQ data
case = 'TRQ'
ingredient_id = pd.read_excel(case_data_file+f'{case}/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('TRQ')
# add target-drug
compounds_gene_file = case_data_file+f'{case}/compounds-gene/'   
for i in range(1,27):
    compounds = drugs[i-1]
    # add Target nodes of case
    compounds2tg_df = pd.read_excel(compounds_gene_file+f'{i}-{compounds}.xlsx')
    v = node_drug[compounds]
    gene_symbol = compounds2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2dg_adj[u][v] = 1
TRQ2tg_df = pd.read_excel(case_data_file+f'{case}/TRQ-gene.xlsx')
v = node_drug['TRQ']
gene_symbol = compounds2tg_df['gene symbol'].to_list()
for g in gene_symbol:
    u = node_target[g]
    t2dg_adj[u][v] = 1
# add target-disease
diseases = ['CRKP', 'KP']
icds = ['[ICD-11: MG50.54]', '[ICD-11: CA40.03]']
for dis in diseases:
    i = diseases.index(dis)
    v = node_icd[icds[i]]
    dis2tg_df = pd.read_excel(case_data_file+f'{case}/disease-gene-{dis}.xlsx')
    gene_symbol = dis2tg_df['gene symbol'].to_list()
    for g in gene_symbol:
        u = node_target[g]
        t2icd_adj[u][v] = 1
# add drug-disease
for dg in drugs:
    u = node_drug[dg]
    for dis in icds:
        v = node_icd[dis]
        dg2d_adj[u][v] = 1

data = {"TvsD":t2icd_adj, "TvsI":t2dg_adj, "IvsD":dg2d_adj, "TvsPK":t2p_adj}

completed: target2pathway_Reactome.npy
completed: target2pathway_KEGG.npy
completed: target2pathway_all.npy
completed: target2drug.npy
completed: target2disease.npy
completed: drug2disease.npy


In [11]:
import dgl
from dgl.data.utils import save_graphs

G = dgl.heterograph({
        ('target1', 'included-by', 'pathway') : data['TvsPK'].nonzero(),
        ('pathway', 'includes', 'target2') : data['TvsPK'].transpose().nonzero(),
        ('target2', 'causes', 'disease') : data['TvsD'].nonzero(),
        ('drug', 'aims-at', 'target1') : data['TvsI'].transpose().nonzero(),
        ('drug', 'treats', 'disease') : data['IvsD'].nonzero(),
    },
    num_nodes_dict = {'disease': len(node_icd), 'drug': len(node_drug),
                      'target1': len(node_target), 'target2': len(node_target),
                      'pathway': len(node_pathway)}
    )

save_graphs(f'./datasets/graph_AP+TRQ', G)
print(G)

Graph(num_nodes={'disease': 1560, 'drug': 38497, 'pathway': 2125, 'target1': 5241, 'target2': 5241},
      num_edges={('drug', 'aims-at', 'target1'): 4579, ('drug', 'treats', 'disease'): 3649, ('pathway', 'includes', 'target2'): 7595, ('target1', 'included-by', 'pathway'): 7595, ('target2', 'causes', 'disease'): 51995},
      metagraph=[('drug', 'target1', 'aims-at'), ('drug', 'disease', 'treats'), ('target1', 'pathway', 'included-by'), ('pathway', 'target2', 'includes'), ('target2', 'disease', 'causes')])


In [16]:
# Genenrat the query pair nids

import torch

query_nids_save_path = './query_nids'
## CQCQD-AP
drugs_nids = []
diseases_nids = []
ingredient_id = pd.read_excel('./data/case/AP/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('CQCQD')
for dg in drugs:
    drugs_nids.append(node_drug[dg])
# add target-disease
icds = ['[ICD-11: DC31]']   # diseases = ['AP']
for dis in icds:
    diseases_nids.append(node_icd[dis])
query_src_nids, query_tgt_nids = torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)
for tgt_nid in diseases_nids:
    for src_nid in drugs_nids:
        query_src_nids = torch.cat((query_src_nids, torch.tensor([src_nid])), dim=0)
        query_tgt_nids = torch.cat((query_tgt_nids, torch.tensor([tgt_nid])), dim=0)
torch.save((query_src_nids, query_tgt_nids), query_nids_save_path+'AP_nids')

## TRQ-CRKP
drugs_nids = []
diseases_nids = []
ingredient_id = pd.read_excel('./data/case/TRQ/ingredient-ttd-drugbank.xlsx')
ingredients = ingredient_id['ingredient name'].tolist()
drugs = sorted(ingredients)
drugs.append('TRQ')
for dg in drugs:
    drugs_nids.append(node_drug[dg])
# add target-disease
icds = ['[ICD-11: MG50.54]', '[ICD-11: CA40.03]']   # diseases = ['CRKP', 'KP']
for dis in icds:
    diseases_nids.append(node_icd[dis])
query_src_nids, query_tgt_nids = torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long)
for tgt_nid in diseases_nids:
    for src_nid in drugs_nids:
        query_src_nids = torch.cat((query_src_nids, torch.tensor([src_nid])), dim=0)
        query_tgt_nids = torch.cat((query_tgt_nids, torch.tensor([tgt_nid])), dim=0)
torch.save((query_src_nids, query_tgt_nids), query_nids_save_path+'TRQ_nids')

##### Drug-ICD11 (gt)

In [ ]:
# import scipy.misc
# from scipy import io
import numpy as np
import pandas as pd
import hdf5storage as hg
import torch

# drug_id2name_dict
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)

# target_id2name_dict
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)

# disease_icd2name_dict
disease_name2icd_file = './data/disease2icd.pkl'
disease_name2icd_dict = read_dict(disease_name2icd_file)

# edges_target2pathway
target2pathway_file = f'./data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
t2p_adj = np.zeros((len(node_target), len(node_pathway)))
for key in target2pathway_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key        
    u = node_target[target]
    for pathway in target2pathway_dict[key]:
        v = node_pathway[pathway]
        t2p_adj[u][v] = 1
# np.save(edges_path+f'target2pathway.npy', t2p_adj)
print(f'completed: target2pathway.npy')

    
# edges_target2drug
target2drug_file = './data/target2drug.xlsx'
t2dg_df = pd.read_excel(target2drug_file)
t2dg_adj = np.zeros((len(node_target), len(node_drug)))
for index, data in t2dg_df.iterrows():
    tid = data[0]    
    drug_id = data[1]
    if tid in target_id2name_dict:
        target = target_id2name_dict[tid]
    else:
        target = tid
    if drug_id in drug_id2name_dict:
        drug = drug_id2name_dict[drug_id]
        u = node_target[target]
        v = node_drug[drug]
        t2dg_adj[u][v] = 1
# np.save(edges_path+'target2drug.npy', t2dg_adj)
print('completed: target2drug.npy')


# edges_target2icd_gt
target2disease_file = './data/groundtruth/gt_target2disease.pkl'
target2disease_dict = read_dict(target2disease_file)
t2icd_adj = np.zeros((len(node_target), len(node_icd)))
for key in target2disease_dict:
    if key in target_id2name_dict:
        target = target_id2name_dict[key]
    else:
        target = key
    u = node_target[target]
    for disease in target2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        t2icd_adj[u][v] = 1    
# np.save(edges_path+'target2disease.npy', t2d_adj)
print('completed: target2disease.npy')


# edges_drug2disease_gt
drug2disease_file = './data/groundtruth/gt_drug2disease.pkl'
drug2disease_dict = read_dict(drug2disease_file)
dg2d_adj = np.zeros((len(node_drug), len(node_icd)))
for key in drug2disease_dict:
    drug = key
    u = node_drug[drug]
    for disease in drug2disease_dict[key]:
        icd = '[' + disease.split(' [')[1].strip()
        v = node_icd[icd]
        dg2d_adj[u][v] = 1
# np.save(edges_path+'drug2disease.npy', dg2d_adj)
print('completed: drug2disease.npy')

# edges_drug2disease_gt_with_labels
dg2dis_label_file = './datasets/graph_AP+TRQ_pred_pair_to_path_labels'
dg2dis_label = torch.load(dg2dis_label_file)
dg2d_gt_adj = np.zeros((len(node_drug), len(node_icd)))
for key in dg2dis_label.keys():
    (scr_ntype, scr_nid), (tgt_ntype, tgt_nid) = key
    u = scr_nid
    v = tgt_nid
    dg2d_gt_adj[u][v] = 1
# np.save(edges_path+'drug2disease.npy', dg2d_gt_adj)
print('completed: drug2disease_gt.npy')


graph_path = './data/graph/mat/'
hg.savemat(graph_path+'graph_groundtruth.mat', {"TvsD":t2icd_adj, "TvsI":t2dg_adj, "IvsD":dg2d_adj, "gtIvsD":dg2d_gt_adj,
                                                 "TvsP":t2p_adj})
print('completed: matlab file')


#### DGL Graph

##### plot schema

In [4]:
# Schema
import pygraphviz as pgv
import dgl

graph_saving_path = './datasets/graph_AP+TRQ'
graph_list, _ = dgl.load_graphs(graph_saving_path)
G = graph_list[0]

def plot_graph(nxg):
    ag = pgv.AGraph(strict=False, directed=True)
    for u, v, k in nxg.edges(keys=True):
        ag.add_edge(u, v, label=k)
    ag.layout('dot')
    ag.draw('./results/schema_graph.png')

plot_graph(G.metagraph())

##### find mAP/sAP eid

In [ ]:
import torch
import dgl

mAP = 'mild acute pancreatitis [ICD-11: DC31]'
sAP = 'severe acute pancreatitis [ICD-11: DC31]'
formula = 'ChaiQinChengQiTang'

graph_saving_path = './datasets/graph_AP+TRQ' 
graph_list, _ = dgl.load_graphs(graph_saving_path)
G = graph_list[0]
print(G)

u, v = G.edges(etype='treats')
M = u.shape[0]
eids = torch.arange(M)
nid = []

for i in eids:
    if int(u[i]) == node_drug[formula] and int(v[i]) == node_disease[mAP]:
        nid.append(i)
    if int(u[i]) == node_drug[formula] and int(v[i]) == node_disease[sAP]:
        nid.append(i)

print(M,nid)

##### Characteristics of G

In [8]:
import torch
import dgl
graph_saving_path = f'./datasets/graph_AP+TRQ'
graph_list, _ = dgl.load_graphs(graph_saving_path)
g = graph_list[0]
# 遍历节点类型，统计节点总数
total_nodes = 0
for ntype in g.ntypes:
    num_node = g.num_nodes(ntype)
    print(ntype, ':', num_node)
    total_nodes += num_node
print('total #nodes:', total_nodes, '\n')
# 遍历边类型，统计边总数
total_edges = 0
for etype in g.etypes:
    num_edge = g.num_edges(etype)
    print(etype, ':', num_edge)
    total_edges += num_edge
print('total #edges:', total_edges, '\n')
# 遍历每条边，统计节点和边的类型
for srctype, etype, dsttype in g.canonical_etypes:
    edge_N = g.num_edges(etype)
    u, v = g.edges(etype=etype)
    src_MN = torch.unique(u).shape[0]
    tgt_MN = torch.unique(v).shape[0]

    src_N = g.num_nodes(srctype)
    tgt_N = g.num_nodes(dsttype)  
    
    print(f'{etype}_{edge_N}: {srctype}:{src_MN}, {dsttype}:{tgt_MN}.  total_num: {srctype}_{src_N}, {dsttype}_{tgt_N}')

disease : 1560
drug : 38497
pathway : 2125
target1 : 5241
target2 : 5241
total #nodes: 52664 

aims-at : 4579
treats : 3649
includes : 7595
included-by : 7595
causes : 51995
total #edges: 75413 

aims-at_4579: drug:2071, target1:1245.  total_num: drug_38497, target1_5241
treats_3649: drug:3347, disease:759.  total_num: drug_38497, disease_1560
includes_7595: pathway:387, target2:1218.  total_num: pathway_2125, target2_5241
included-by_7595: target1:1218, pathway:387.  total_num: target1_5241, pathway_2125
causes_51995: target2:2958, disease:747.  total_num: target2_5241, disease_1560


# Labels

In [1]:
import pickle

def file_read(file):
    with open(file) as f:
        data = f.readlines()
    return data

def save_dict(dict, file_path):
    f_save = open(file_path, 'wb')
    pickle.dump(dict, f_save)
    f_save.close()

def read_dict(file_path):
    f_read = open(file_path, 'rb')
    dict = pickle.load(f_read)
    f_read.close()
    return dict

In [22]:
import torch
import pandas as pd
from collections import defaultdict
import dgl

nodes_path = './data/graph/nodes/'
node_target = read_dict(nodes_path+'target_dict.pkl')  # gene symbol
node_drug = read_dict(nodes_path+'drug_dict.pkl')  # drug_name
node_pathway = read_dict(nodes_path+'pathway_dict.pkl')  # Reactome_id
node_disease = read_dict(nodes_path+'disease_dict.pkl')  # Indication_ICD-11
node_icd11 = read_dict(nodes_path+'icd11_dict.pkl')  # icd11_code
# drug_id2name_dict
drug_id2name_file = './data/drug_ttdid2name.pkl'
drug_id2name_dict = read_dict(drug_id2name_file)
# target_id2name_dict
target_id2name_file = './data/target_ttdid2name.pkl'
target_id2name_dict = read_dict(target_id2name_file)

# Target - Pathway
target2pathway_file = './data/target2pathway_KEGG.pkl'
target2pathway_dict = read_dict(target2pathway_file)
# "Target - Disease" GroundTruth
target2disease_file = './data/groundtruth/gt_target2disease.pkl'
target2disease = read_dict(target2disease_file)
# "Disease - Target" 
dis2tg_dic = {}
for tid, diseases in target2disease.items():
    if tid in target_id2name_dict:        
        tg = target_id2name_dict[tid]
    else:
        tg = tid
    for dis in diseases:
        icd = '[' + dis.split(' [')[1].strip()
        if icd not in dis2tg_dic:
            dis2tg_dic[icd] = []
        if tg not in dis2tg_dic[icd]:
            dis2tg_dic[icd].append(tg)

# Drug - Target
target2drug_file = './data/target2drug.xlsx'
t2dg_df = pd.read_excel(target2drug_file)
dg2tg_dic = {}
for index, data in t2dg_df.iterrows():
    tid = data[0]    
    drug_id = data[1]
    if tid in target_id2name_dict:
        tg = target_id2name_dict[tid]
    else:
        tg = tid
    if drug_id in drug_id2name_dict:
        dg = drug_id2name_dict[drug_id]
        if dg not in dg2tg_dic:
            dg2tg_dic[dg] = []
        if tg not in dg2tg_dic[dg]:
            dg2tg_dic[dg].append(tg)

# # groundtruth: "Drug - Disease"
# drug2disease_file = './data/groundtruth/gt_drug2disease.pkl'
# dg2dis_dict = read_dict(drug2disease_file)
# '''
# # add case: CQCQD-AP, TRQ-CRKP/KP
# case_drugs = ['CQCQD', 'TRQ']
# case_diseases = [['AP [ICD-11: DC31]'], ['CRKP [ICD-11: MG50.54]', 'KP [ICD-11: CA40.03]']]   # diseases = ['AP', ['CRKP', 'KP']]
# for i in range(len(case_drugs)):
#     dg = case_drugs[i]
#     dises = case_diseases[i]
#     if dg not in dg2dis_dict:
#         dg2dis_dict[dg] = dises
#     else:
#         for dis in dises:
#             if dis not in dg2dis_dict[dg]:
#                 dg2dis_dict[dg].append(icd)
# '''

# __init__: edge_label_dict, path_label_dict
dg2tg = ('drug', 'aims-at', 'target1')
tg2p = ('target1', 'included-by', 'pathway')
p2tg = ('pathway', 'includes', 'target2')
tg2dis = ('target2', 'causes', 'disease')
path_label_dict = defaultdict(list)
edge_label_dict = {}
# graph_saving_path = './datasets/graph_AP+TRQ'
# graph_list, _ = dgl.load_graphs(graph_saving_path)
# g = graph_list[0]
# u,v = g.edges(etype='treats')
# src_N = g.num_nodes('drug')
# tgt_N = g.num_nodes('disease')
# M = u.shape[0]
# for i in range(M):
#     dg_id = int(u[i])
#     dis_id = int(v[i])
#     gt_pair = (('drug', dg_id), ('disease', dis_id))
#     if gt_pair not in edge_label_dict:
#         edge_label_dict[gt_pair] = defaultdict(set)
#         edge_label_dict[gt_pair][dg2tg] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
#         edge_label_dict[gt_pair][tg2p] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
#         edge_label_dict[gt_pair][p2tg] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
#         edge_label_dict[gt_pair][tg2dis] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
# print(len(edge_label_dict))
# pred_pair_to_path/edge_labels

# groundtruth: "Drug - Disease"
drug2disease_file = './data/groundtruth/gt_drug2disease.pkl'
dg2dis_dict = read_dict(drug2disease_file)
for dg, diseases in dg2dis_dict.items():
    for dis in diseases:
        icd = '[' + dis.split(' [')[1].strip()
        gt_pair = (('drug', node_drug[dg]), ('disease', node_icd11[icd]))
        if dg in dg2tg_dic and icd in dis2tg_dic:
            ''''''
            if gt_pair not in edge_label_dict:
                edge_label_dict[gt_pair] = defaultdict(set)
                edge_label_dict[gt_pair][dg2tg] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
                edge_label_dict[gt_pair][tg2p] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
                edge_label_dict[gt_pair][p2tg] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
                edge_label_dict[gt_pair][tg2dis] = (torch.tensor([], dtype=torch.long), torch.tensor([], dtype=torch.long))
            ''''''
            tg_list1 = dg2tg_dic[dg]
            tg_list2 = dis2tg_dic[icd]
            '''
            # dg-tg-dis
            intersection = set(tg_list1) & set(tg_list2)
            for tg in intersection:  
                e1 = (torch.tensor(node_drug[dg]), torch.tensor(node_target[tg]))
                e2 = (torch.tensor(node_target[tg]), torch.tensor(node_icd11[icd]))
                p1 = (dg2tg, node_drug[dg], node_target[tg])
                p2 = (tg2dis, node_target[tg], node_icd11[icd])
                path = [p1,p2]
                if path not in path_label_dict[gt_pair]:
                    path_label_dict[gt_pair].append(path)
                    # edge_label_dict[gt_pair][dg2tg]
                    src, dst = edge_label_dict[gt_pair][dg2tg]
                    src = torch.cat((src, torch.tensor([e1[0]])), dim=0)
                    dst = torch.cat((dst, torch.tensor([e1[1]])), dim=0)
                    edge_label_dict[gt_pair][dg2tg] = (src.to(torch.int64), dst.to(torch.int64))
                    # edge_label_dict[gt_pair][tg2dis]
                    src, dst = edge_label_dict[gt_pair][tg2dis]
                    src = torch.cat((src, torch.tensor([e2[0]])), dim=0)
                    dst = torch.cat((dst, torch.tensor([e2[1]])), dim=0)
                    edge_label_dict[gt_pair][tg2dis] = (src.to(torch.int64), dst.to(torch.int64)) 
            '''
            # dg-tg-p-tg-dis
            for tg1 in tg_list1: # dg2tg_dic[dg]
                if tg1 in target2pathway_dict:
                    pathway_set1 = target2pathway_dict[tg1]
                else:
                    pathway_set1 = []
                for tg2 in tg_list2: # dis2tg_dic[dis]
                    if tg2 in target2pathway_dict:
                        pathway_set2 = target2pathway_dict[tg2]
                    else:
                        pathway_set2 = []
                    intersection = set(pathway_set1) & set(pathway_set2)            
                    
                    for pathway in intersection: 
                        e1 = (torch.tensor(node_drug[dg]), torch.tensor(node_target[tg1]))
                        e2 = (torch.tensor(node_target[tg1]), torch.tensor(node_pathway[pathway]))
                        e3 = (torch.tensor(node_pathway[pathway]), torch.tensor(node_target[tg2]))
                        e4 = (torch.tensor(node_target[tg2]), torch.tensor(node_icd11[icd]))
                        p1 = (dg2tg, node_drug[dg], node_target[tg1])
                        p2 = (tg2p, node_target[tg1], node_pathway[pathway])
                        p3 = (p2tg, node_pathway[pathway], node_target[tg2])
                        p4 = (tg2dis, node_target[tg2], node_icd11[icd])
                        path = [p1,p2,p3,p4]
                        if path not in path_label_dict[gt_pair]:
                            path_label_dict[gt_pair].append(path)
                            # edge_label_dict[gt_pair][dg2tg]
                            src, dst = edge_label_dict[gt_pair][dg2tg]
                            src = torch.cat((src, torch.tensor([e1[0]])), dim=0)
                            dst = torch.cat((dst, torch.tensor([e1[1]])), dim=0)
                            edge_label_dict[gt_pair][dg2tg] = (src.to(torch.int64), dst.to(torch.int64))
                            # edge_label_dict[gt_pair][tg2p]
                            src, dst = edge_label_dict[gt_pair][tg2p]
                            src = torch.cat((src, torch.tensor([e2[0]])), dim=0)
                            dst = torch.cat((dst, torch.tensor([e2[1]])), dim=0)
                            edge_label_dict[gt_pair][tg2p] = (src.to(torch.int64), dst.to(torch.int64))
                            # edge_label_dict[gt_pair][p2tg]
                            src, dst = edge_label_dict[gt_pair][p2tg]
                            src = torch.cat((src, torch.tensor([e3[0]])), dim=0)
                            dst = torch.cat((dst, torch.tensor([e3[1]])), dim=0)
                            edge_label_dict[gt_pair][p2tg] = (src.to(torch.int64), dst.to(torch.int64))
                            # edge_label_dict[gt_pair][tg2dis]
                            src, dst = edge_label_dict[gt_pair][tg2dis]
                            src = torch.cat((src, torch.tensor([e4[0]])), dim=0)
                            dst = torch.cat((dst, torch.tensor([e4[1]])), dim=0)
                            edge_label_dict[gt_pair][tg2dis] = (src.to(torch.int64), dst.to(torch.int64)) 
                    paths_num = len(path_label_dict[gt_pair])
            if gt_pair not in path_label_dict.keys():
                edge_label_dict.pop(gt_pair, None)
print(len(path_label_dict), len(edge_label_dict))
torch.save(path_label_dict,'./datasets/graph_AP+TRQ_pred_pair_to_path_labels')
torch.save(edge_label_dict,'./datasets/graph_AP+TRQ_pred_pair_to_edge_labels')

1674 1674 0 451 4221 5241


Read label files

In [7]:
import torch

f1 = './datasets/graph_AP+TRQ_pred_pair_to_path_labels'
f2 = './datasets/graph_AP+TRQ_pred_pair_to_edge_labels'
pred_pair_to_edge_labels = torch.load(f1)
pred_pair_to_path_labels = torch.load(f2)

# Retrieve explanation results into txt

In [ ]:
'''
target = []  # TTD_target_id
drug = []  # drug_name
pathway = []  # Reactome_id
disease = []  # Indication_ICD-11
icd = []    # ICD-11
herb = []  # HERB_herb_id
'''
node_dict = {}
nodes_path = './data/graph/nodes/'
node_dict['disease'] = retrieve_dict(read_dict(nodes_path+'icd11_dict.pkl'))
node_dict['pathway'] = retrieve_dict(read_dict(nodes_path+'pathway_dict.pkl'))
node_dict['drug'] = retrieve_dict(read_dict(nodes_path+'drug_dict.pkl'))
node_dict['target'] = retrieve_dict(read_dict(nodes_path+'target_dict.pkl'))
node_dict['target1'] = retrieve_dict(read_dict(nodes_path+'target_dict.pkl'))
node_dict['target2'] = retrieve_dict(read_dict(nodes_path+'target_dict.pkl'))

# node_id2name = {}
# node_id2name['target'] = read_dict('./data/target_ttdid2name.pkl')

num_path = 50
pathwaySrc = 'pathway'
# preprocess = 'DEG'   # ML, RWR, DEG
dataset_name = 'graph_AP+TRQ' # f'woHERB_KEGG_case_{preprocess}'
explanations_file = './code/saved_explanations/'
query_case = 'AP'
dic = read_dict(explanations_file+f'{query_case}_{num_path}.pkl')

case = dic.keys()
print(len(case))

SaveList=[]
for dis in case:
    (src_type, src_nid), (tgt_type, tgt_nid) = dis
    src = node_dict[src_type][src_nid]
    tgt = node_dict[tgt_type][tgt_nid]
    pair = (src, tgt)
    SaveList.append(str(pair))
    for paths in dic[dis]:
        line = []
        for path in paths:
            edge, src_nid, tgt_nid = path
            src_type, edge_type, tgt_type = edge            
            src = node_dict[src_type][src_nid]
            tgt = node_dict[tgt_type][tgt_nid]
            
            # if tgt_type == 'disease':
            #     tgt = icd2dis[tgt]    

            if src_type == 'drug' and len(line)==0:
                line += [src, edge_type, tgt]
            else:
                line += [edge_type, tgt]
        # print(str(line))
        SaveList.append(str(line))
    SaveList.append('\n')

results_file = './results/'
save_txt = results_file + f'{query_case}_{num_path}.txt'
with open(save_txt,'w',encoding='utf-8') as file:
    for i in SaveList:
        file.write(i+'\n')
    file.close       